In [ ]:
#MAHIDI HASAN MITHUN
#ID : 1930432

import numpy as np
import matplotlib.pyplot as plt
import random

def data(file_path):
    cities = []
    with open(file_path, 'r') as file:
        lines = file.readlines()
        coord_section = False
        for line in lines:
            if coord_section:
                if line.strip() == 'EOF':
                    break
                city_info = line.split()
                cities.append((float(city_info[1]), float(city_info[2])))
            elif line.strip() == 'NODE_COORD_SECTION':
                coord_section = True
        return cities

def opt_tour(file_path):
    tour = []
    with open(file_path, 'r') as file:
        lines = file.readlines()
        tour_section = False
        for line in lines:
            if tour_section:
                if line.strip() == '-1':
                    break
                city_idx = int(line.strip()) - 1
                tour.append(city_idx)
            elif line.strip() == 'TOUR_SECTION':
                tour_section = True
        return tour

def distance(city1, city2):
    return np.sqrt((city1[0] - city2[0])**2 + (city1[1] - city2[1])**2)

def initialize_population(pop_size, num_cities):
    population = []
    for _ in range(pop_size):
        individual = np.random.permutation(num_cities)
        population.append(individual)
    return population

def cal_fitness(population, cities):
    fitness = []
    for individual in population:
        dist = 0
        for i in range(len(individual) - 1):
            dist += distance(cities[individual[i]], cities[individual[i + 1]])
        dist += distance(cities[individual[-1]], cities[individual[0]])
        fitness.append(1 / dist)
    return fitness

def select_parents(population, fitness):
    total_fitness = np.sum(fitness)
    probabilities = fitness / total_fitness
    parents = np.random.choice(len(population), size=len(population), p=probabilities)
    return parents

def crossover(parent1, parent2):
    crossover_point = np.random.randint(1, len(parent1) - 1)
    child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
    child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
    return child1, child2

def mutate(individual, mutation_rate):
    for i in range(len(individual)):
        if np.random.rand() < mutation_rate:
            j = np.random.randint(len(individual))
            individual[i], individual[j] = individual[j], individual[i]
    return individual

def genetic_algorithm(cities, pop_size, num_generations, mutation_rate, optimal_tour):
    num_cities = len(cities)
    population = initialize_population(pop_size, num_cities)
    best_distances = []

    for gen in range(num_generations):
        fitness = cal_fitness(population, cities)
        parents = select_parents(population, fitness)

        new_population = []
        for i in range(0, len(parents), 2):
            parent1 = population[parents[i]]
            parent2 = population[parents[i + 1]]
            child1, child2 = crossover(parent1, parent2)
            child1 = mutate(child1, mutation_rate)
            child2 = mutate(child2, mutation_rate)
            new_population.extend([child1, child2])

        population = new_population

        best_idx = np.argmax(fitness)
        best_distance = 1 / fitness[best_idx]
        best_distances.append(best_distance)

    best_individual = population[best_idx]
    best_route = [cities[i] for i in best_individual]

    return best_route, best_distances

def calculate_relative_error(best_distance, optimal_distance):
    return ((best_distance - optimal_distance) / optimal_distance) * 100

def main():
    files = ['berlin52', 'att48', 'pr76']
    optimal_files = ['berlin52.opt.txt', 'att48.opt.txt', 'pr76.opt.txt']



    for file_name, optimal_file in zip(files, optimal_files):
        city_data = data(f"{file_name}.txt")
        optimal_tour = opt_tour(optimal_file)
        optimal_distance = sum(distance(city_data[optimal_tour[i]], city_data[optimal_tour[i + 1]]) for i in range(len(optimal_tour) - 1))


        #pop_size = 50
        pop_size = 100
        #pop_size = 250
        #pop_size = 500
        num_generations = 100
        mutation_rate = 0.1

        print(f"Solving TSP : {file_name}")
        best_route, best_distances = genetic_algorithm(city_data, pop_size, num_generations, mutation_rate, optimal_tour)

        best_distance = sum(distance(best_route[i], best_route[i + 1]) for i in range(len(best_route) - 1))
        error_rel = calculate_relative_error(best_distance, optimal_distance)

        print(f"Optimal Distance: {optimal_distance:.2f}")
        print(f"Best Distance: {best_distance:.2f}")
        print(f"Relative Error: {error_rel:.2f}%")

        plt.plot(best_distances)
        plt.xlabel("Generation")
        plt.ylabel("Best Distance")
        plt.title(f"Genetic Algorithm for TSP - {file_name}")
        plt.show()

if __name__ == "__main__":
    main()
